In [5]:
import uproot
import numpy as np #math and science package
import scipy as sp #math and science package
import awkward as ak #root files are usuallt awkward arrays 
import matplotlib.pyplot as plt #plot stuff
import itertools
from my_functions import *

#Open the file
file= uproot.open("/home/ruben/Escritorio/BachelorThesisRuben/Data/Muon_trees.root")

#Get both keys
MuonTree_Zmumu=file["MuonTree_Zmumu;1"]
MuonTree_ZeroBias=file["MuonTree_ZeroBias;1"] 

#Get eta, phi and pt
eta=MuonTree_Zmumu["muon_eta"].array()
phi=MuonTree_Zmumu["muon_phi"].array()
pt=MuonTree_Zmumu["muon_pt"].array()

#Now let's select the best pairs 
pairs=[]
for i in range(100):
    m=pair_selector(pt, eta, phi, i)
    pairs.append(m[1])

In [7]:
ak_element_lenght_counter(pairs)

<Array [(3, 3), (3, 3), (...), ..., (3, 3), (3, 3)] type='100 * (int64, int64)'>